<a href="https://colab.research.google.com/github/jerpint/jerpint.github.io/blob/master/colabs/reserve_climb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Blog post: https://www.jerpint.io/blog/find-climb-spots/

For emails to work, you will need to create an account on sendgrid and export your API keys as an environment variable, e.g.:

`source ~/sendgrid.env`


In [ ]:
from datetime import date
import requests
import time

import pandas as pd
from sendgrid import SendGridAPIClient
from sendgrid.helpers.mail import Mail


def send_email_func(
    html_content, from_email, to_emails, subject,
):
    message = Mail(
        from_email=from_email,
        to_emails=to_emails,
        subject=subject,
        html_content=html_content,
    )
    try:
        sg = SendGridAPIClient(os.environ.get("SENDGRID_API_KEY"))
        response = sg.send(message)
        print(response.status_code)
        print(response.body)
        print(response.headers)
    except Exception as e:
        print(e.message)


def extract_availabilities(resp):
    """
    Convert the response to a dict:
        availabilities = {
            time_slot (str): is_available (bool)
    }
    """

    # extract the availabilities raw html table and convert it to a pandas dataframe
    raw_fields = pd.read_html(resp.json()["event_list_html"])[0]

    # pandas voodoo to extract dates and availabilities
    processed_fields = raw_fields.apply(
        lambda x: {
            x[0].split("to")[0].split(",")[2].strip(" "): x[1]
            in ["AvailabilityAvailable", "Availability2 spaces"]
        },
        axis=1,
    ).values

    # combine final results into a single dict
    availabilities = {}
    for y in processed_fields:
        availabilities = {**availabilities, **y}

    return availabilities


def check_available_slots(availabilities, time_slots, date_str):
    """Check if desired time slots are available, returns first available one and the email contents.""" 

    spot_found = False
    email_content = ""
    for slot in time_slots:
        assert (
            slot in availabilities
        ), f"Check the time format {slot} you're requesting."
        if availabilities[slot]:
            spot_found = True
            email_content = f"Theres a spot available at the time you wanted, {slot} on {date_str}. Go book it now."
            break

    return email_content


def fetch_data(date_str):
    """
    Fetch the raw data from the website hosting the information.
    
    date_str (str): day to climb at, expressed in format YYYY-MM-DD
    """
    url = "https://app.rockgympro.com/b/widget/?a=equery"
    
    # this data is required for the post request, not sure what all of it is but fails without it
    data = {
        "show_date": date_str,
        "PreventChromeAutocomplete": "",
        "random": "60feea89d7a01",
        "iframeid": "rgpiframe60feea88d990c",
        "mode": "e",
        "fctrl_1": "offering_guid",
        "offering_guid": "fc473e424ba64f0b9bf5063dd07054a1",
        "fctrl_2": "course_guid",
        "course_guid": "",
        "fctrl_3": "limited_to_course_guid_for_offering_guid_fc473e424ba64f0b9bf5063dd07054a1",
        "limited_to_course_guid_for_offering_guid_fc473e424ba64f0b9bf5063dd07054a1": "",
        "fctrl_4": "show_date",
        "ftagname_0_pcount-pid-1-7746024": "pcount",
        "ftagval_0_pcount-pid-1-7746024": "1",
        "ftagname_1_pcount-pid-1-7746024": "pid",
        "ftagval_1_pcount-pid-1-7746024": "7746024",
        "fctrl_5": "pcount-pid-1-7746024",
        "pcount-pid-1-7746024": "0",
        "ftagname_0_pcount-pid-1-7746303": "pcount",
        "ftagval_0_pcount-pid-1-7746303": "1",
        "ftagname_1_pcount-pid-1-7746303": "pid",
        "ftagval_1_pcount-pid-1-7746303": "7746303",
        "fctrl_6": "pcount-pid-1-7746303",
        "pcount-pid-1-7746303": "0",
        "ftagname_0_pcount-pid-1-7754428": "pcount",
        "ftagval_0_pcount-pid-1-7754428": "1",
        "ftagname_1_pcount-pid-1-7754428": "pid",
        "ftagval_1_pcount-pid-1-7754428": "7754428",
        "fctrl_7": "pcount-pid-1-7754428",
        "pcount-pid-1-7754428": "0",
    }
    return requests.post(url, data=data)


def find_climbing_spots(date, time_slots, refresh_rate=20, send_email=True):
    """
    Find a climbining spot on a given date for given times.
    
    date (datetime.date): Day, month, year in a datetime format
    time_slots list[(str)]: Times of day to climb at e.g. ['7 PM', '7:30 PM']
    refresh_rate (int): Time (in seconds) to wait before refreshing the page again
    seng_email (bool): Send an email if a spot is found
    """

    date_str = f"{date.year:4d}-{date.month:02d}-{date.day:02d}"
    count = 0
    spot_found = False
    while not spot_found:
        # Only refresh the page once every $refresh_rate seconds, avoid spamming
        if count > 0:
            time.sleep(refresh_rate)

        try:
            resp = fetch_data(date_str)
        except requests.exceptions.RequestException:
            print("Something went wrong fetching the page. Will try again...")
            continue

        if resp.ok:
            availabilities = extract_availabilities(resp)
            email_content = check_available_slots(
                availabilities, time_slots, date_str
            )
            
        spot_found = False if email_content == "" else True 

        if spot_found:
            print(email_content)
            if send_email:
                send_email_func(
                    html_content=email_content,
                    subject="Nice, you can climb!",
                    from_email="jerpint@gmail.com",
                    to_emails=["jerpint@gmail.com"],
                )
                print("\nEmail sent!")
        else:
            count += 1
            print(f"Refreshing in {refresh_rate} seconds. Refresh count: {count}", end="\r")


find_climbing_spots(
    date=date(year=2021, month=7, day=29),
    time_slots=["6 PM", "6:30 PM", "7 PM", "7:30 PM",],  
    refresh_rate=30,
    send_email=True,
)